In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
import re
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
import keras

In [35]:
# csv 파일을 dataframe으로 변환
df_outfit = pd.read_csv('../data/outfit(female)/outfit(female).csv')
df_weather = pd.read_csv('../data/2022-08-01_to_2024-04-17.csv', encoding='cp949')
# 필요한 columns만 추출
df_outfit = df_outfit[['userId', '상의', '아우터', '하의', '신발', '엑세서리', '작성일']].copy()
df_temp = df_weather[['일시', '평균기온(°C)']].copy()

# '작성일'과 '일시' 열을 datetime 형식으로 변환
df_outfit['작성일'] = pd.to_datetime(df_outfit['작성일'], format='%Y년 %m월 %d일')
df_temp['일시'] = pd.to_datetime(df_temp['일시'])

# 두 dataframe을 날짜를 기준으로 병합
df_merged = pd.merge(df_outfit, df_temp, left_on='작성일', right_on='일시').drop('일시', axis=1)

df_merged

,userId,상의,아우터,하의,신발,엑세서리,작성일,평균기온(°C)
0,1.0,민소매 티,재킷,트레이닝/조거 팬츠,운동화,NaN,2023-02-06,3.0
1,1.0,민소매 티,집업,반바지,운동화,NaN,2023-02-18,5.7
2,1.0,맨투맨,NaN,트레이닝/조거 팬츠,운동화,NaN,2023-03-20,9.5
3,1.0,반팔 티,집업,반바지,운동화,NaN,2023-04-20,15.8
4,1.0,반팔 티,집업,트레이닝/조거 팬츠,운동화,NaN,2023-04-22,16.9
...,...,...,...,...,...,...,...,...
224,6.0,셔츠/블라우스,NaN,데님팬츠,구두/로퍼,NaN,2023-10-12,17.5
225,6.0,맨투맨,NaN,미니/미디스커트,운동화,장목양말,2023-10-24,15.6
226,6.0,"맨투맨, 후드티",NaN,미니/미디스커트,레더부츠,장목양말,2023-10-25,16.6
227,6.0,긴팔 티,재킷,미니/미디스커트,운동화,스타킹,2023-11-01,19.0


In [36]:
# '상의', '아우터', '하의', '신발', '엑세서리' 열의 결측값을 '~ 없음'으로 대체
columns = ['상의', '아우터', '하의', '신발', '엑세서리']
df_notnull = df_merged.copy()
for column in columns:
    df_notnull[column] = df_merged[column].fillna(column + ' 없음')

In [37]:
# 2가 붙은 단어를 두 번 반복하는 함수
def duplicate_word(text):
    words = text.split(', ')
    for i, word in enumerate(words):
        if '2' in word:
            words[i] = word.replace('2', '') + ', ' + word.replace('2', '')
    return ', '.join(words)

In [38]:
# 2가 붙은 단어를 두 번 반복한 dataframe df_dup 생성
df_dup = df_notnull.copy()
for column in columns:
    df_dup[columns] = df_notnull[columns].map(duplicate_word)

In [39]:
df_dup[df_dup['아우터'] == '재킷 , 재킷 ']

,userId,상의,아우터,하의,신발,엑세서리,작성일,평균기온(°C)
80,3.0,반팔 티,"재킷 , 재킷",트레이닝/조거 팬츠,스니커즈/캔버스,엑세서리 없음,2024-03-13,6.5
84,3.0,반팔 티,"재킷 , 재킷",데님팬츠,스니커즈/캔버스,기타 모자,2024-03-20,4.4
86,3.0,반팔 티,"재킷 , 재킷",데님팬츠,구두/로퍼,엑세서리 없음,2024-03-28,10.3


In [40]:
df_dup.columns

Index(['userId', '상의', '아우터', '하의', '신발', '엑세서리', '작성일', '평균기온(°C)'], dtype='object')

In [41]:
# 옷의 조합 컬럼 생성 (상의, 아우터, 하의, 신발, 엑세서리의 각 값들을 하나의 문자열로 조합하여 하나의 컬럼으로 만듦)
df_combination = df_dup.copy()
df_combination['옷 조합'] = df_dup['상의'] + ', ' + df_dup['아우터'] + ', ' + df_dup['하의'] + ', ' + df_dup['신발'] + ', ' + df_dup['엑세서리']
df_combination.drop(columns=['상의', '아우터', '하의', '신발', '엑세서리'], inplace=True)

In [42]:
df_combination.columns

Index(['userId', '작성일', '평균기온(°C)', '옷 조합'], dtype='object')

In [43]:
'''df_combination[df_combination['옷 조합'].str.contains('니트 , 니트')]'''

"df_combination[df_combination['옷 조합'].str.contains('니트 , 니트')]"

In [44]:
# 옷의 조합 컬럼의 공백 제거
df_combination['옷 조합'] = df_combination['옷 조합'].str.replace(' ', '')

In [45]:
# 쉼표를 기준으로 텍스트를 나누는 함수
def comma_tokenizer(s):
    return s.split(',')

vectorizer = CountVectorizer(tokenizer=comma_tokenizer)

O = vectorizer.fit_transform(df_combination['옷 조합'])

# multi-hot encoding된 데이터를 numpy array로 변환
df_encoded = pd.DataFrame(O.toarray().tolist(), columns=vectorizer.get_feature_names_out())
npa = np.array(df_encoded)
npa.shape

/home/ksy/anaconda3/envs/ondoset/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(229, 53)

In [46]:
# 값이 2 이상인 행의 인덱스
rows_with_value_2 = df_encoded[(df_encoded >= 2).any(axis=1)]
rows_with_value_2.index

Index([80, 84, 86, 137], dtype='int64')

In [47]:
# 값이 2 이상인 열의 이름을 찾습니다.
columns_with_value_over_2 = df_encoded.columns[(df_encoded >= 2).any()]

# 특정 행에 대해 이를 기록합니다.
record = df_encoded.loc[rows_with_value_2.index, columns_with_value_over_2]
record

,니트,재킷
80,0,2
84,0,2
86,0,2
137,2,0


In [48]:
# 단어장 확인
vectorizer.get_feature_names_out()

array(['가디건', '가죽바지', '구두/로퍼', '기타모자', '긴팔티', '니삭스', '니트', '데님팬츠', '레깅스',
       '레더부츠', '레인부츠', '롱스커트', '롱원피스', '롱패딩', '맨투맨', '머플러', '면바지',
       '미니/미디스커트', '미니/미디원피스', '민소매티', '바람막이', '반바지', '반팔니트', '반팔셔츠/블라우스',
       '반팔티', '비니', '상의없음', '샌들/슬리퍼', '셔츠/블라우스', '스니커즈/캔버스', '스카프', '스타킹',
       '슬랙스', '신발없음', '아우터없음', '어그부츠', '엑세서리없음', '운동화', '장갑', '장목양말',
       '재킷', '점퍼', '조끼', '집업', '카고바지', '코트', '털모자', '트레이닝/조거팬츠', '패딩',
       '패딩조끼', '하의없음', '후드티', '힐'], dtype=object)

In [49]:
# numpy array를 list로 변환 후 clothes_combination 컬럼에 대입
df_combination['옷 조합'] = npa.tolist()
df_combination['옷 조합']

0      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
3      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                             ...                        
224    [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...
225    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
226    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...
227    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
228    [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...
Name: 옷 조합, Length: 229, dtype: object

In [51]:
# multi-hot encoding된 데이터를 다시 텍스트로 변환
df_combination['옷 조합'] = vectorizer.inverse_transform(npa)
df_combination['옷 조합'] 

0           [민소매티, 엑세서리없음, 운동화, 재킷, 트레이닝/조거팬츠]
1                 [민소매티, 반바지, 엑세서리없음, 운동화, 집업]
2         [맨투맨, 아우터없음, 엑세서리없음, 운동화, 트레이닝/조거팬츠]
3                  [반바지, 반팔티, 엑세서리없음, 운동화, 집업]
4            [반팔티, 엑세서리없음, 운동화, 집업, 트레이닝/조거팬츠]
                        ...                   
224      [구두/로퍼, 데님팬츠, 셔츠/블라우스, 아우터없음, 엑세서리없음]
225          [맨투맨, 미니/미디스커트, 아우터없음, 운동화, 장목양말]
226    [레더부츠, 맨투맨, 미니/미디스커트, 아우터없음, 장목양말, 후드티]
227              [긴팔티, 미니/미디스커트, 스타킹, 운동화, 재킷]
228          [긴팔티, 데님팬츠, 스니커즈/캔버스, 엑세서리없음, 재킷]
Name: 옷 조합, Length: 229, dtype: object

In [52]:
# 하나의 문자열로 변환
df_combtest = df_combination.copy()
df_combtest['옷 조합'] = df_combination['옷 조합'].apply(lambda x: ', '.join(map(str, x)))

In [53]:
# multi-hot encoding의 값이 2 이상인 경우, 해당 단어를 두 번 반복
for i in record.index:
    old_value = df_combtest.loc[i, '옷 조합']
    for col in record.columns:
        if record.loc[i, col] >= 2:
            old_value = old_value.replace(col, col + ', ' + col)
    df_combtest.loc[i, '옷 조합'] = old_value

In [54]:
df_combtest.loc[rows_with_value_2.index, '옷 조합']

80     반팔티, 스니커즈/캔버스, 엑세서리없음, 재킷, 재킷, 트레이닝/조거팬츠
84            기타모자, 데님팬츠, 반팔티, 스니커즈/캔버스, 재킷, 재킷
86             구두/로퍼, 데님팬츠, 반팔티, 엑세서리없음, 재킷, 재킷
137              니트, 니트, 레더부츠, 롱스커트, 엑세서리없음, 코트
Name: 옷 조합, dtype: object

In [55]:
# 중복된 행 찾기
duplicated_rows = df_combtest[df_combtest.duplicated(['옷 조합'], keep=False)]
duplicated_rows.sort_values(by='옷 조합')

,userId,작성일,평균기온(°C),옷 조합
17,1.0,2024-02-28,6.4,"가디건, 데님팬츠, 민소매티, 엑세서리없음, 운동화"
20,1.0,2024-04-05,14.0,"가디건, 데님팬츠, 민소매티, 엑세서리없음, 운동화"
178,5.0,2023-10-11,17.6,"구두/로퍼, 니트, 면바지, 아우터없음, 엑세서리없음"
151,5.0,2023-02-13,5.9,"구두/로퍼, 니트, 면바지, 아우터없음, 엑세서리없음"
208,6.0,2023-09-03,25.8,"구두/로퍼, 데님팬츠, 셔츠/블라우스, 아우터없음, 엑세서리없음"
224,6.0,2023-10-12,17.5,"구두/로퍼, 데님팬츠, 셔츠/블라우스, 아우터없음, 엑세서리없음"
119,4.0,2023-05-27,19.3,"구두/로퍼, 면바지, 셔츠/블라우스, 엑세서리없음, 재킷"
105,4.0,2023-04-12,10.1,"구두/로퍼, 면바지, 셔츠/블라우스, 엑세서리없음, 재킷"
5,1.0,2023-05-25,21.2,"기타모자, 반팔티, 샌들/슬리퍼, 아우터없음, 트레이닝/조거팬츠"
13,1.0,2023-07-26,27.6,"기타모자, 반팔티, 샌들/슬리퍼, 아우터없음, 트레이닝/조거팬츠"


In [56]:
# pivot_table을 이용한 user-item matrix 생성|
UI_matrix = df_combtest.pivot_table(values='평균기온(°C)', index='userId', columns='옷 조합', fill_value=0)
UI_matrix.columns

Index(['가디건, 구두/로퍼, 니트, 엑세서리없음, 트레이닝/조거팬츠', '가디건, 기타모자, 민소매티, 운동화, 트레이닝/조거팬츠',
       '가디건, 기타모자, 민소매티, 운동화, 트레이닝/조거팬츠, 패딩', '가디건, 니트, 스니커즈/캔버스, 슬랙스, 엑세서리없음',
       '가디건, 데님팬츠, 민소매티, 엑세서리없음, 운동화', '가디건, 데님팬츠, 반팔티, 스니커즈/캔버스, 엑세서리없음',
       '가디건, 레더부츠, 롱원피스, 엑세서리없음, 하의없음', '가디건, 레더부츠, 미니/미디원피스, 스타킹, 하의없음',
       '가디건, 롱원피스, 샌들/슬리퍼, 장목양말, 하의없음', '가디건, 반바지, 반팔티, 스니커즈/캔버스, 엑세서리없음',
       ...
       '반바지, 셔츠/블라우스, 스니커즈/캔버스, 아우터없음, 장목양말',
       '반바지, 셔츠/블라우스, 스니커즈/캔버스, 엑세서리없음, 재킷',
       '반바지, 셔츠/블라우스, 스니커즈/캔버스, 장목양말, 재킷',
       '반팔티, 스니커즈/캔버스, 엑세서리없음, 재킷, 재킷, 트레이닝/조거팬츠',
       '반팔티, 아우터없음, 엑세서리없음, 운동화, 카고바지', '반팔티, 아우터없음, 엑세서리없음, 운동화, 트레이닝/조거팬츠',
       '반팔티, 엑세서리없음, 운동화, 집업, 트레이닝/조거팬츠',
       '샌들/슬리퍼, 아우터없음, 엑세서리없음, 트레이닝/조거팬츠, 후드티',
       '스니커즈/캔버스, 아우터없음, 엑세서리없음, 트레이닝/조거팬츠, 후드티',
       '스니커즈/캔버스, 엑세서리없음, 집업, 트레이닝/조거팬츠, 후드티'],
      dtype='object', name='옷 조합', length=212)

In [75]:
# pivot_table을 이용한 user_
UI_count = df_combtest.pivot_table( index='userId', columns='옷 조합', aggfunc='size', fill_value=0) 
UI_count

옷 조합,"가디건, 구두/로퍼, 니트, 엑세서리없음, 트레이닝/조거팬츠","가디건, 기타모자, 민소매티, 운동화, 트레이닝/조거팬츠","가디건, 기타모자, 민소매티, 운동화, 트레이닝/조거팬츠, 패딩","가디건, 니트, 스니커즈/캔버스, 슬랙스, 엑세서리없음","가디건, 데님팬츠, 민소매티, 엑세서리없음, 운동화","가디건, 데님팬츠, 반팔티, 스니커즈/캔버스, 엑세서리없음","가디건, 레더부츠, 롱원피스, 엑세서리없음, 하의없음","가디건, 레더부츠, 미니/미디원피스, 스타킹, 하의없음","가디건, 롱원피스, 샌들/슬리퍼, 장목양말, 하의없음","가디건, 반바지, 반팔티, 스니커즈/캔버스, 엑세서리없음",...,"반바지, 셔츠/블라우스, 스니커즈/캔버스, 아우터없음, 장목양말","반바지, 셔츠/블라우스, 스니커즈/캔버스, 엑세서리없음, 재킷","반바지, 셔츠/블라우스, 스니커즈/캔버스, 장목양말, 재킷","반팔티, 스니커즈/캔버스, 엑세서리없음, 재킷, 재킷, 트레이닝/조거팬츠","반팔티, 아우터없음, 엑세서리없음, 운동화, 카고바지","반팔티, 아우터없음, 엑세서리없음, 운동화, 트레이닝/조거팬츠","반팔티, 엑세서리없음, 운동화, 집업, 트레이닝/조거팬츠","샌들/슬리퍼, 아우터없음, 엑세서리없음, 트레이닝/조거팬츠, 후드티","스니커즈/캔버스, 아우터없음, 엑세서리없음, 트레이닝/조거팬츠, 후드티","스니커즈/캔버스, 엑세서리없음, 집업, 트레이닝/조거팬츠, 후드티"
userId,,,,,,,,,,,,,,,,,,,,,
1.0,0,1,1,0,2,0,0,0,0,1,...,0,0,0,0,2,1,1,0,0,0
2.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
3.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4.0,1,0,0,1,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
5.0,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
6.0,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [59]:
UI_matrix.to_csv('../data/outfit(female)/user_item_matrix.csv', encoding='utf-8-sig')

In [60]:
# user-item matrix에 기록된 값이 존재하는 경우 1, 아닌 경우 0으로 변환하여 R_df에 기록
R_df = UI_matrix.map(lambda x: 1 if x != 0 else 0).copy()
R_np = np.array(R_df)
R_np.sum(axis=0)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [62]:
# 각 열의 합이 2 이상(여러 유저가 해당 옷 조합을 선택한 경우)인 열을 찾아 출력
columns_with_sum_over_2 = R_df.columns[R_df.sum() >= 2]
'''R_df[columns_with_sum_over_2]'''

'R_df[columns_with_sum_over_2]'

In [63]:
'''# 해당 조합의 인덱스
column_index = []
for i in columns_with_sum_over_2:
    column_index.append(R_df.columns.get_loc(i))
    column_index
column_index'''

'# 해당 조합의 인덱스\ncolumn_index = []\nfor i in columns_with_sum_over_2:\n    column_index.append(R_df.columns.get_loc(i))\n    column_index\ncolumn_index'

In [64]:
'''for row in R_np:
    print(row)'''

'for row in R_np:\n    print(row)'

In [65]:
# CF를 위한 초기값 설정
Y = np.array(UI_matrix)
Y = Y.T
count = np.array(UI_count)
count = count.T
print(Y.shape)
R = Y != 0 
n_u = Y.shape[1]
n_o = Y.shape[0]

(212, 6)


In [66]:
# 기록이 존재하는 값의 평균을 구함
o_sum = Y.sum(axis=1)
o_count = R.sum(axis=1)
o_mean = o_sum / o_count
o_mean = o_mean.reshape(-1, 1)
o_mean_df = pd.DataFrame(o_mean, index=UI_matrix.columns)
o_mean_df

,0
옷 조합,
"가디건, 구두/로퍼, 니트, 엑세서리없음, 트레이닝/조거팬츠",11.8
"가디건, 기타모자, 민소매티, 운동화, 트레이닝/조거팬츠",7.1
"가디건, 기타모자, 민소매티, 운동화, 트레이닝/조거팬츠, 패딩",5.8
"가디건, 니트, 스니커즈/캔버스, 슬랙스, 엑세서리없음",19.0
"가디건, 데님팬츠, 민소매티, 엑세서리없음, 운동화",10.2
...,...
"반팔티, 아우터없음, 엑세서리없음, 운동화, 트레이닝/조거팬츠",22.2
"반팔티, 엑세서리없음, 운동화, 집업, 트레이닝/조거팬츠",16.9
"샌들/슬리퍼, 아우터없음, 엑세서리없음, 트레이닝/조거팬츠, 후드티",4.7


In [67]:
Y_stand = Y - (o_mean * R)
'''Y_stand[column_index]'''

'Y_stand[column_index]'

In [68]:
def cofi_cost_func_v(O, U, b, Y, R, lambda_):
    j = (tf.linalg.matmul(O, tf.transpose(U)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(O**2) + tf.reduce_sum(U**2))
    return J

In [69]:
#  Useful Values
n_o, n_u = Y.shape
num_features = 30

# Set Initial Parameters (W, X), use tf.Variable to track these variables
tf.random.set_seed(1234) # for consistent results
U = tf.Variable(tf.random.normal((n_u,  num_features),dtype=tf.float64),  name='U')
O = tf.Variable(tf.random.normal((n_o, num_features),dtype=tf.float64),  name='O')
b = tf.Variable(tf.random.normal((1,          n_u),   dtype=tf.float64),  name='b')

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

2024-04-24 21:17:07.538225: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-24 21:17:07.629116: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-24 21:17:07.629179: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-24 21:17:07.634780: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-24 21:17:07.634835: I external/local_xla/xla/stream_executor

In [70]:
J = cofi_cost_func_v(O, U, b, Y_stand, R, 1.5)

2024-04-24 21:17:10.319424: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [71]:
print(f"Cost (with regularization): {J:0.2f}")

Cost (with regularization): 7916.24


In [72]:
iterations = 200
lambda_ = 1
for iter in range(iterations):
    # TensorFlow의 GradientTape 사용
    # 연산을 기록하여 cost에 대한 gradient를 자동으로 계산
    with tf.GradientTape() as tape:

        # cost 계산 (forward pass included in cost)
        cost_value = cofi_cost_func_v(O, U, b, Y_stand, R, lambda_)

    # GradientTape를 통해 자동 미분
    # loss에 대한 trainable parameter의 gradient를 계산
    grads = tape.gradient( cost_value, [O,U,b] )

    # optimizer를 사용하여 trainable parameter를 업데이트
    optimizer.apply_gradients( zip(grads, [O,U,b]) )

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

2024-04-24 21:17:22.264023: I external/local_xla/xla/service/service.cc:168] XLA service 0xb222cd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-24 21:17:22.264074: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-04-24 21:17:22.274783: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-24 21:17:22.296917: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1713961042.364000     827 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Training loss at iteration 0: 6307.2
Training loss at iteration 20: 460.4
Training loss at iteration 40: 87.1
Training loss at iteration 60: 27.7
Training loss at iteration 80: 15.1
Training loss at iteration 100: 11.2
Training loss at iteration 120: 9.4
Training loss at iteration 140: 8.4
Training loss at iteration 160: 7.8
Training loss at iteration 180: 7.5


In [82]:
# 예측을 수행하기 위해 모든 user-item에 대한 예측값을 계산
p = np.matmul(O.numpy(), np.transpose(U.numpy())) + b.numpy()
# 실제 온도
temp = 25
# 평균을 적용하고 temp를 빼서 값이 작을수록 실제 온도에 가깝도록 함. 이 때 각 user-item의 사용 횟수를 가중하여 많이 사용한 item이 추천되도록 함
pm = np.power(p + o_mean - temp, 2)  -count * 0.5

# 각 user마다 반복
for i in range(n_u) :
    my_predictions = pm[:,i]

    # sort predictions
    ix = tf.argsort(my_predictions, direction='ASCENDING')

    df_predict = UI_matrix[UI_matrix.columns[ix[0:10]]]
    df_predict.to_csv(f'../data/predictions/female/user_{i+1}_predictions.csv')

In [83]:
df_predict

옷 조합,"맨투맨, 반바지, 스니커즈/캔버스, 아우터없음, 장목양말","긴팔티, 레인부츠, 반바지, 아우터없음, 장목양말","반바지, 반팔티, 스니커즈/캔버스, 아우터없음, 장목양말","기타모자, 민소매티, 아우터없음, 운동화, 트레이닝/조거팬츠","데님팬츠, 셔츠/블라우스, 스니커즈/캔버스, 아우터없음, 엑세서리없음","레더부츠, 롱원피스, 반팔티, 아우터없음, 엑세서리없음, 하의없음","반바지, 샌들/슬리퍼, 셔츠/블라우스, 아우터없음, 엑세서리없음","면바지, 반팔니트, 스니커즈/캔버스, 아우터없음, 엑세서리없음","기타모자, 반팔티, 샌들/슬리퍼, 아우터없음, 트레이닝/조거팬츠","기타모자, 맨투맨, 미니/미디스커트, 반팔티, 아우터없음, 운동화, 장목양말"
userId,,,,,,,,,,
1.0,0.0,0.0,0.0,24.7,0.0,0.0,0.0,0.0,24.4,0.0
2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,0.0,0.0,25.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,0.0,0.0,0.0,0.0,24.7,0.0,0.0,0.0,0.0,0.0
5.0,0.0,0.0,0.0,0.0,0.0,25.2,24.6,25.4,0.0,0.0
6.0,24.7,24.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0


In [86]:
UI_count['긴팔티, 레인부츠, 반바지, 아우터없음, 장목양말']

userId
1.0    0
2.0    0
3.0    0
4.0    0
5.0    0
6.0    1
Name: 긴팔티, 레인부츠, 반바지, 아우터없음, 장목양말, dtype: int64